## AI Agent Leveraging Functin Apps for Tool Calling

In [1]:
from dotenv import load_dotenv  
import os  
from azure.monitor.opentelemetry import configure_azure_monitor
  
# Load the .env file  
load_dotenv(override=True) 

True

In [2]:
import json
import datetime
from typing import Any, Callable, Set, Dict, List, Optional

def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location: The location to fetch weather for.
    :return: Weather information as a JSON string.
    """
    
    print("location:", location)

    import requests

    url = "https://func.azurewebsites.net/api/http_trigger"
    params = {"code": "blah", "location": location}

    response = requests.get(url, params=params)

    # Print the response
    print("Status Code:", response.status_code)
    print("Response Body:", response.text)

    return response.text


# Define user functions
user_functions = {fetch_weather}

In [3]:

async def find_or_create_agent(agent_list, agent_name, logger):
    agent = None
    if agent_list:
        async for agent_object in agent_list:
            if agent_object.name == agent_name:
                agent = agent_object
                logger.info(f"Found agent by name '{agent_name}', ID={agent_object.id}")
                return agent
    
        if not agent:
            logger.info(f"Agent with name '{agent_name}' not found, creating a new agent.")
            return None


In [4]:
import os, time
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FunctionTool
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder
import logging
import asyncio


# Configure logger
import logging
import sys

# Create a logger
logger = logging.getLogger("notebook_logger")
logger.setLevel(logging.INFO)

# Clear existing handlers to avoid duplicate logs
if logger.hasHandlers():
    logger.handlers.clear()

# Create handler that outputs to notebook
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger.addHandler(handler)

# Example usage
logger.info("Logger is now printing to the notebook!")



# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]

# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'
# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()

logger.info(f"Application Insights connection string: {application_insights_connection_string}")

if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your Azure AI Foundry project page.")
    exit()
    
configure_azure_monitor(connection_string=application_insights_connection_string)

agent_name = os.environ["AZURE_AI_AGENT_NAME"]
agent_list = project_client.agents.list_agents()

agent = None
if agent_list:
    for agent_object in agent_list:
        if agent_object.name == agent_name:
            agent = agent_object

if not agent:
    logger.info(f"Agent with name '{agent_name}' not found, creating a new agent.")
    functions = FunctionTool({fetch_weather})
    toolset = ToolSet()
    toolset.add(functions)
    project_client.agents.enable_auto_function_calls(toolset)

        # Create an agent with custom functions

    agent = project_client.agents.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"],
            name=agent_name,
            instructions="You are a helpful agent with tools to fetch weather information.",
            tools=functions.definitions,
        )
    logger.info(f"Created agent, ID: {agent.id}")
else:
    logger.info(f"Found agent by name '{agent_name}', ID={agent.id}")

2025-07-03 12:07:04,444 - INFO - Logger is now printing to the notebook!
2025-07-03 12:07:08,144 - INFO - Application Insights connection string: InstrumentationKey=ff987441-70b7-44fc-aa34-15451599ad0e;IngestionEndpoint=https://eastus2-3.in.applicationinsights.azure.com/;LiveEndpoint=https://eastus2.livediagnostics.monitor.azure.com/;ApplicationId=82c61bb8-3b97-4b35-8985-4009623618f8
2025-07-03 12:07:14,058 - INFO - Found agent by name 'mmx-ai-agent05', ID=asst_qBptgH0scFOlmmdLd381Ogqc


In [5]:
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content="what is the weather in London?",  # Message content
    )
print(f"Created message, ID: {message['id']}")

Created thread, ID: thread_HG6SHWIV5SxWmKueW9HAa5rN
Created message, ID: msg_rXvgV9fviEbdIq11yAGsjfgH


In [6]:
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

print(agent.name)
print(agent.id)

Run finished with status: RunStatus.COMPLETED
mmx-ai-agent05
asst_qBptgH0scFOlmmdLd381Ogqc


In [7]:
run

{'id': 'run_XoaqY7rURhv6l7ZElhNR6SqM', 'object': 'thread.run', 'created_at': 1751562433, 'assistant_id': 'asst_qBptgH0scFOlmmdLd381Ogqc', 'thread_id': 'thread_HG6SHWIV5SxWmKueW9HAa5rN', 'status': 'completed', 'started_at': 1751562436, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1751562437, 'required_action': None, 'last_error': None, 'model': 'gpt-4.1', 'instructions': 'You are a helpful agent with tools to fetch weather information.', 'tools': [{'type': 'function', 'function': {'name': 'fetch_weather', 'description': 'Fetches the weather information for the specified location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to fetch weather for.'}}, 'required': ['location']}, 'strict': False}}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'inco

## View your Message

In [8]:
if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message.role}, Content: {message.content}")

Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': "I'm unable to retrieve the weather for London at this moment due to a system issue. Would you like to try again, or do you want information on typical weather conditions in London instead?", 'annotations': []}}]
Role: MessageRole.USER, Content: [{'type': 'text', 'text': {'value': 'what is the weather in London?', 'annotations': []}}]


## View your Tool Calls

In [9]:
run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
        # print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  Tool calls:")
            for call in tool_calls:
                print(call)
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")
                type = call.get("type")
                if type == "function":
                    print(f"    Function Name: {call.get('function', {}).get('name')}")
                    print(f"    Function Arguments: {call.get('function', {}).get('arguments')}")
              

  Tool calls:
{'id': 'call_zZPCjezldSBH8BkqVKfq5ShU', 'type': 'function', 'function': {'name': 'fetch_weather', 'arguments': '{"location":"London"}', 'output': '{"error": "Error executing function \'fetch_weather\': Function \'fetch_weather\' not found. Provide this function to `enable_auto_function_calls` function call."}'}}
    Tool Call ID: call_zZPCjezldSBH8BkqVKfq5ShU
    Type: function
    Function Name: fetch_weather
    Function Arguments: {"location":"London"}


In [10]:
from azure.ai.evaluation import CoherenceEvaluator, AzureOpenAIModelConfiguration

model_config = AzureOpenAIModelConfiguration(
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_key=os.environ.get("AZURE_OPENAI_KEY"),
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
        api_version=os.environ.get("AZURE_API_VERSION"),
    )


## Check Coherence

In [11]:
# Extract user and agent messages for coherence evaluation
coherence = CoherenceEvaluator(model_config=model_config, threshold=3)
user_message = None
agent_message = None
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(message.role)
    if str(message.role) == "MessageRole.USER" and user_message is None:
        user_message = message.content
    elif str(message.role) in ("MessageRole.AGENT") and agent_message is None:
        agent_message = message.content

if user_message and agent_message:
    print("checking coherence")
    coherence_score = coherence(query=user_message, response=agent_message)
    print(f"Coherence score: {coherence_score}")
else:
    print("Could not find both user and agent messages for coherence evaluation.")

MessageRole.AGENT
MessageRole.USER
checking coherence
2025-07-03 12:07:31 -0500   27968 azure.ai.evaluation._legacy.prompty._prompty ERROR    [0/10] AsyncAzureOpenAI request failed. NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}
Traceback (most recent call last):
  File "c:\Users\memasanz\repos\ai-foundry-agent-func\.venv\Lib\site-packages\azure\ai\evaluation\_legacy\prompty\_prompty.py", line 377, in _send_with_retries
    response = await client.chat.completions.create(**params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\memasanz\repos\ai-foundry-agent-func\.venv\Lib\site-packages\openai\resources\chat\completions\completions.py", line 2028, in create
    return await self._post(
           ^^^^^^^^^^^^^^^^^
    ...<45 lines>...
    )
    ^
  File "c:\Users\memasa

WrappedOpenAIError: (UserError) OpenAI API hits NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}} [Error reference: https://platform.openai.com/docs/guides/error-codes/api-errors]

## Check Intent Resolution

In [ ]:
from azure.ai.evaluation import IntentResolutionEvaluator

intent_resolution = IntentResolutionEvaluator(model_config=model_config, threshold=3)
intent_resolution(
    query=user_message,
    response=agent_message
)

## QA Evaluator

In [ ]:
from azure.ai.evaluation import QAEvaluator

qa_eval = QAEvaluator(model_config=model_config, threshold=3)
qa_eval(
    query="Where was Marie Curie born?", 
    context="Background: 1. Marie Curie was a chemist. 2. Marie Curie was born on November 7, 1867. 3. Marie Curie is a French scientist.",
    response="According to wikipedia, Marie Curie was not born in Paris but in Warsaw.",
    ground_truth="Marie Curie was born in Warsaw."
)